In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [1]:
# example of pix2pix gan for satellite to map image-to-image translation
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from numpy import asarray
import numpy as np
from numpy import vstack
import tensorflow as tf
from keras import backend as K
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from matplotlib import pyplot
from os import listdir
import cv2

AttributeError: module 'tensorflow.python.keras.utils.generic_utils' has no attribute 'populate_dict_with_module_objects'

In [2]:
print(tf.__version__)

NameError: name 'tf' is not defined

In [ ]:
# dice
def dice_coef(y_true, y_pred, k=1):
    #intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    #union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    #return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)
    
    #intersection = np.sum(y_pred[y_true==k]) * 2.0
    #dice = intersection / (np.sum(y_pred) + np.sum(y_true))
    #return dice
    
    y_true_f = tf.reshape(tf.dtypes.cast(y_true, tf.float32), [-1])
    y_pred_f = tf.reshape(tf.dtypes.cast(y_pred, tf.float32), [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + 1.) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + 1.)

# IoU
def iou(y_true, y_pred, smooth=100):
    #intersection = np.logical_and(y_true, y_pred)
    #union = np.logical_or(y_true, y_pred)
    #iou_score = np.sum(intersection) / np.sum(union)
    #return iou_score
    '''y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    union = K.sum(y_true_f) + K.sum(y_pred_f) - intersection
    iou_ = intersection/union
    return iou_'''

    """ Calculates mean of Jaccard distance as a loss function """
    intersection = tf.reduce_sum(y_true * y_pred, axis=(1,2))
    sum_ = tf.reduce_sum(y_true + y_pred, axis=(1,2))
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    jd =  (1 - jac)
    return tf.reduce_mean(jd)
    
# accuracy
def metrics(y_true, y_pred):
    #print(y_pred.shape,y_true.shape,np.unique(y_pred),np.unique(y_true))
    cm = confusion_matrix(y_pred.ravel(),y_true.ravel())
    # tn -> Verdeiro Negativo
    # fp -> Falso Positivo
    # fn -> Falso Negativo
    # tp -> Verdadeiro Positivo
    #print(cm.ravel().shape)
    #print(y_true.shape, " ", y_pred.shape)
    tn, fp, fn, tp = cm.ravel()
    dice = (2.0 * tp) / ((2.0 * tp) + fp + fn)
    #jaccard = (1.0 * tp) / (tp + fp + fn)
    #sensitivity = (1.0 * tp) / (tp + fn)
    #specificity = (1.0 * tn) / (tn + fp)
    accuracy = (1.0 * (tn + tp)) / (tn + fp + tp + fn)
    #auc = 1 - 0.5 * (((1.0 * fp) / (fp + tn)) + ((1.0 * fn) / (fn + tp)))
    #prec = float(tp)/float(tp + fp)
    #fscore = float(2*tp)/float(2*tp + fp + fn)
    #kappa = cohen_kappa_score(y_pred, y_true)
    #return sensitivity, specificity, accuracy, auc, kappa #, dice
    return accuracy

In [ ]:
# define the discriminator model
def define_discriminator(image_shape):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # source image input
    in_src_image = Input(shape=image_shape)
    # target image input
    in_target_image = Input(shape=image_shape)
    # concatenate images channel-wise
    merged = Concatenate()([in_src_image, in_target_image])
    # C64
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    # C128
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C256
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C512
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # second last output layer
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # patch output
    d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(d)
    # define model
    model = Model([in_src_image, in_target_image], patch_out)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    #model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5], metrics=[dice_coef, iou, "accuracy"])
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model

In [ ]:
# define an encoder block
def define_encoder_block(layer_in, n_filters, batchnorm=True):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # add downsampling layer
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    # conditionally add batch normalization
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    # leaky relu activation
    g = LeakyReLU(alpha=0.2)(g)
    return g

In [ ]:
# define a decoder block
def decoder_block(layer_in, skip_in, n_filters, dropout=True):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # add upsampling layer
    g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    # add batch normalization
    g = BatchNormalization()(g, training=True)
    # conditionally add dropout
    if dropout:
        g = Dropout(0.5)(g, training=True)
    # merge with skip connection
    g = Concatenate()([g, skip_in])
    # relu activation
    g = Activation('relu')(g)
    return g

In [ ]:
# define the standalone generator model
def define_generator(image_shape=(256,256,3)):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # image input
    in_image = Input(shape=image_shape)
    # encoder model
    e1 = define_encoder_block(in_image, 64, batchnorm=False)
    e2 = define_encoder_block(e1, 128)
    e3 = define_encoder_block(e2, 256)
    e4 = define_encoder_block(e3, 512)
    e5 = define_encoder_block(e4, 512)
    e6 = define_encoder_block(e5, 512)
    e7 = define_encoder_block(e6, 512)
    # bottleneck, no batch norm and relu
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
    b = Activation('relu')(b)
    # decoder model
    d1 = decoder_block(b, e7, 512)
    d2 = decoder_block(d1, e6, 512)
    d3 = decoder_block(d2, e5, 512)
    d4 = decoder_block(d3, e4, 512, dropout=False)
    d5 = decoder_block(d4, e3, 256, dropout=False)
    d6 = decoder_block(d5, e2, 128, dropout=False)
    d7 = decoder_block(d6, e1, 64, dropout=False)
    # output
    g = Conv2DTranspose(3, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
    out_image = Activation('tanh')(g)
    # define model
    model = Model(in_image, out_image)
    return model

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model, image_shape):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # define the source image
    in_src = Input(shape=image_shape)
    # connect the source image to the generator input
    gen_out = g_model(in_src)
    # connect the source input and generator output to the discriminator input
    dis_out = d_model([in_src, gen_out])
    # src image as input, generated image and classification output
    model = Model(in_src, [dis_out, gen_out])
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    #model.compile(loss=['binary_crossentropy'], optimizer=opt, loss_weights=[1,100], metrics=[dice_coef, iou, "accuracy"])
    return model

In [4]:
# load all images in a directory into memory
def load_images(path, size=(256,512)):
    src_list, tar_list = list(), list()
    # enumerate filenames in directory, assume all are images
    for filename in listdir(path):
        # load and resize the image
        pixels = load_img(path + filename, target_size=size)
        # convert to numpy array
        pixels = img_to_array(pixels)
        # split into ct and mask
        ct_img, mask_img = pixels[:, :256], pixels[:, 256:]
        
        #bin_mask
        #mask_img = (mask_img > 128) * 1
        
        src_list.append(ct_img)
        tar_list.append(mask_img)
    return [asarray(src_list), asarray(tar_list)]
 
# dataset path
path = '/data/flavio/anatiel/datasets/AB_lung_lesion/img-lung-lesion/train/'
# load dataset
[src_images, tar_images] = load_images(path)
print('Loaded: ', src_images.shape, tar_images.shape)
# save as compressed numpy array
#filename = 'maps_256.npz'
#savez_compressed(filename, src_images, tar_images)
#print('Saved dataset: ', filename)

Loaded:  (1474, 256, 256, 3) (1474, 256, 256, 3)


In [ ]:
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples, patch_shape):
    # unpack dataset
    trainA, trainB = dataset
    # choose random instances
    ix = randint(0, trainA.shape[0], n_samples)
    # retrieve selected images
    X1, X2 = trainA[ix], trainB[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, patch_shape, patch_shape, 1))
    return [X1, X2], y

In [ ]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, patch_shape):
    # generate fake instance
    X = g_model.predict(samples)    
    # create 'fake' class labels (0)
    y = zeros((len(X), patch_shape, patch_shape, 1))
    return X, y

In [ ]:
def generate_metrics(target, pred):
    return accuracy_score(target, pred)

In [ ]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, dataset, n_samples=3):
    # select a sample of input images
    [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
    # generate a batch of fake samples
    X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
    # scale all pixels from [-1,1] to [0,1]
    X_realA = (X_realA + 1) / 2.0
    X_realB = (X_realB + 1) / 2.0
    X_fakeB = (X_fakeB + 1) / 2.0
    # plot real source images
    for i in range(n_samples):
        pyplot.subplot(3, n_samples, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(X_realA[i], cmap="gray")
    # plot generated target image
    for i in range(n_samples):
        pyplot.subplot(3, n_samples, 1 + n_samples + i)
        pyplot.axis('off')
        pyplot.imshow(X_fakeB[i], cmap="gray")
    # plot real target image
    for i in range(n_samples):
        pyplot.subplot(3, n_samples, 1 + n_samples*2 + i)
        pyplot.axis('off')
        pyplot.imshow(X_realB[i], cmap="gray")
        
    # save plot to file
    filename1 = '/outputs/plot_%06d.png' % (step+1)
    pyplot.savefig(filename1)
    pyplot.close()
    # save the generator model
    filename2 = '/models/model_%06d.h5' % (step+1)
    g_model.save(filename2)
    print('>Saved: %s and %s' % (filename1, filename2))    

In [ ]:
# create a line plot of loss for the gan and save to file
def plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist, a3_hist):
    pyplot.figure(figsize=(40, 40))
    fig, axs = pyplot.subplots(2, 2)
    # plot loss
    axs[0, 0].plot(d1_hist, label='d-real')
    axs[0, 0].plot(d2_hist, label='d-fake')
    axs[0, 0].plot(g_hist, label='gen')
    axs[0, 0].set(xlabel='Epochs', ylabel='Loss')
    #axs[0, 0].ylabel('Loss')
    #axs[0, 0].xlabel('Epochs')
    axs[0, 0].legend()
    #axs[0, 0].set_title("Discriminator Loss")
    
    # plot discriminator dice
    axs[0, 1].plot(a1_hist, label='gen-dice')
    axs[0, 1].set(xlabel='Epochs', ylabel='Dice')
    #axs[0, 1].ylabel('Dice Coef')
    #axs[0, 1].xlabel('Epochs')
    axs[0, 1].legend()
    #axs[0, 1].set_title("Discriminator Dice")
    
    # plot discriminator IoU
    axs[1, 0].plot(a2_hist, label='gen-iou')
    axs[1, 0].set(xlabel='Epochs', ylabel='IoU')
    #axs[1, 0].ylabel('IoU')
    #axs[1, 0].xlabel('Epochs')
    axs[1, 0].legend()
    #axs[1, 0].set_title("Discriminator IoU")
    
    # plot discriminator accuracy
    axs[1, 1].plot(a3_hist, label='gen-acc')
    axs[1, 1].set(xlabel='Epochs', ylabel='Accuracy')
    #axs[1, 1].ylabel('Accuracy')
    #axs[1, 1].xlabel('Epochs')
    axs[1, 1].legend()
    #axs[1, 1].set_title("Discriminator Accuracy")
    
    # save plot to file
    pyplot.savefig('/results_baseline/plot_line_plot_loss.png')
    pyplot.close()

In [ ]:
def progress_monitor(epoch, d1_hist, d2_hist, g_hist, a1_hist, a2_hist, a3_hist):
    with open("/home/flavio/anatiel/progress_monitor.txt", "w") as f:
        f.writelines(map("d_loss1 {},d_loss2 {},g_loss {},d_dice {},d_iou {},d_acc {}\n".format, d1_hist, d2_hist, g_hist, a1_hist, a2_hist, a3_hist))

In [ ]:
# train pix2pix models
def train(d_model, g_model, gan_model, dataset, n_epochs=1000, n_batch=1):
    # determine the output square shape of the discriminator
    n_patch = d_model.output_shape[1]
    # unpack dataset
    trainA, trainB = dataset
    
    # calculate the number of batches per training epoch
    bat_per_epo = int(len(trainA) / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo #* n_epochs
    
    '''--------------------------'''
    # prepare lists for storing stats each iteration
    d1_hist, d2_hist, g_hist, a1_hist, a2_hist, a3_hist = list(), list(), list(), list(), list(), list()
    avg_d1_hist, avg_d2_hist, avg_g_hist, avg_a1_hist, avg_a2_hist, avg_a3_hist = list(), list(), list(), list(), list(), list()
    '''--------------------------'''
    
    # manually enumerate epochs
    for epoch in range(n_epochs):
        print("Epoch", epoch+1, "/", n_epochs)
        for i in range(n_steps):
            # select a batch of real samples
            [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
            # generate a batch of fake samples
            X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
            # update discriminator for real samples
            #d_loss1, d_dice1, d_iou1, d_acc1 = d_model.train_on_batch([X_realA, X_realB], y_real)
            d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
            # update discriminator for generated samples
            #d_loss2, d_dice2, d_iou2, d_acc2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
            d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
            # update the generator
            g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
            #g_loss = gan_model.train_on_batch(X_realA, [y_real, X_realB])
            
            
            '''--------------------------'''
            #cv2.imwrite("/data/flavio/anatiel/datasets/covid19-256x256-generated/" + new_name_img, img_resized)
            '''--------------------------'''
            
            #bin_mask
            X_fakeB = (X_fakeB > 128) * 1
            X_realB = (X_realB > 128) * 1
            
            # pegar X_realA(imagem entrada) e X_fakeB(imagem gerada)
            gen_dice = dice_coef(X_realB, X_fakeB)
            #gen_acc = metrics(X_realB, X_fakeB)
            gen_iou = iou(X_realB, X_fakeB)
            
            '''--------------------------'''
            # Accuracy calculated using Keras' method
            metric = tf.keras.metrics.Accuracy()
            metric.update_state(X_realB,X_fakeB.round())
            gen_acc = metric.result().numpy()
            
            print("DICE", gen_dice.numpy(), "IoU", gen_iou.numpy(), "ACC", gen_acc)
            '''--------------------------'''
            
            '''--------------------------'''
            # record history
            d1_hist.append(d_loss1)
            d2_hist.append(d_loss2)
            g_hist.append(g_loss)
            a1_hist.append(gen_dice.numpy())
            a2_hist.append(gen_iou.numpy())
            a3_hist.append(gen_acc)
            '''--------------------------'''
            
            # summarize performance
            if (i+1) % 100 == 0:
                #print('Step %d, d_loss1[%.3f] d_loss2[%.3f] g_loss[%.3f] d_dice1[%.3f] d_dice2[%.3f] d_iou1[%.3f] d_iou2[%.3f] d_acc1[%.3f] d_acc2[%.3f]' % (i+1, d_loss1, d_loss2, g_loss, d_dice1, d_dice2, d_iou1, d_iou2, d_acc1, d_acc2))
                # summarize model performance
                print("Saving model")
                summarize_performance(i, g_model, dataset)
            
        '''--------------------------'''
        # record history average
        avg_d1_hist.append(sum(d1_hist) / len(d1_hist))
        avg_d2_hist.append(sum(d2_hist) / len(d2_hist))
        avg_g_hist.append(sum(g_hist) / len(g_hist))
        avg_a1_hist.append(sum(a1_hist) / len(a1_hist))
        avg_a2_hist.append(sum(a2_hist) / len(a2_hist))
        avg_a3_hist.append(sum(a3_hist) / len(a3_hist))
        print("Epoch Avg: ", avg_d1_hist, avg_d2_hist, avg_g_hist, avg_a1_hist, avg_a2_hist, avg_a3_hist)
        '''--------------------------'''
                
        '''--------------------------'''
        plot_history(avg_d1_hist, avg_d2_hist, avg_g_hist, avg_a1_hist, avg_a2_hist, avg_a3_hist)
        progress_monitor(epoch, avg_d1_hist, avg_d2_hist, avg_g_hist, avg_a1_hist, avg_a2_hist, avg_a3_hist)
        '''--------------------------'''

In [ ]:
# load image data
#dataset = load_real_samples('maps_256.npz')
dataset = [src_images, tar_images]
#n_batch = 1
#n_patch = 16
print('Loaded', dataset[0].shape, dataset[1].shape)
# define input shape based on the loaded dataset
image_shape = dataset[0].shape[1:]

In [ ]:
# define the models
d_model = define_discriminator(image_shape)
g_model = define_generator(image_shape)
# define the composite model
gan_model = define_gan(g_model, d_model, image_shape)

In [ ]:
# train model
train(d_model, g_model, gan_model, dataset)